In [56]:
import spacy
import pickle
import string

In [57]:
from spacy.lang.en import STOP_WORDS

In [58]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [94]:
class Sentiment_Parser:
    """ Wraps up everything related to sentiments in here """
    nlp = spacy.load("en_core_web_sm")
    sia = SentimentIntensityAnalyzer()
    stop_words = set(STOP_WORDS)
    stop_words.update(string.punctuation)
    
    def __init__(self, review):
        self.review = Sentiment_Parser.nlp(review)
        
    def splitter (self):
        self.splits = []
        def _split (sentence):
            start = 0
            counter = 0
            for token in sentence:
                if token.pos == 89 or token.text.strip() == ',' or token.text.strip() == '.':
                    if counter > start:
                        self.splits.append (sentence[start: counter])
                    start = counter + 1
                counter += 1
            if len (sentence[start: counter]) > 0:
                self.splits.append (sentence[start: counter])
        
        for sent in self.review.sents:
            # print ("Sentence: ", sent)
            _split(sent)
            
    def feature_extraction(self):
        self.features = []
        for entry in self.splits:
            nouns = []
            verbs = []
            adjs = []
            
            adverb = None
            
            for t in entry:
                if t.pos == 92 or t.pos == 96:
                    """ If token is a Noun """
                    nouns.append(t)
                elif t.pos == 84:
                    """ If token is an Adjective """
                    ## Prior to adding the Adjectives tokens, we better check if an adverb is there
                    if adverb:
                        adjs.append(str(adverb) + " " + str(t))
                    else:
                        adjs.append(t)
                elif t.pos == 100:
                    """ If token is a Verb """
                    ## Prior to adding the Verb tokens, we better check if an adverb is there
                    if adverb:
                        verbs.append(str(adverb) + " " + str(t))
                    else:
                        verbs.append(t)
                elif t.pos == 94:
                    """ If token is an Adverb, (like : 'not')"""
                    if adverb:
                        adverb += " " + t
                    else:
                        adverb = t
                    
            if len(adjs) == 0:
                adjs = verbs
            self.features.append( (",".join(map(str, nouns)), ",".join(map(str, adjs))) )
            
    def pprint(self):
        self.splitter()
        self.feature_extraction()
        print(f"\tORIGINAL TEXT: {self.review}\n")
        for entry, features in zip(self.splits, self.features):
            print(f"TEXT: {entry}")
            scores = Sentiment_Parser.sia.polarity_scores(str(entry))
            
            if scores['compound'] > 0:
                result = "POSITIVE"
            elif scores['compound'] < 0:
                result = "NEGATIVE"
            else:
                result = "NEUTRAL"
            
            print(f"RESULT: {result}")
            #print(f"SCORE: {scores}")
            print(f"Entity-Feature pair: {features}")
            print()

In [95]:
if __name__ == '__main__':
    review2 = " I liked the food but service was not good"
    sp = Sentiment_Parser(review2)
    sp.pprint()
    

	ORIGINAL TEXT:  I liked the food but service was not good

TEXT:  I liked the food
RESULT: POSITIVE
Entity-Feature pair: ('food', 'liked')

TEXT: service was not good
RESULT: NEGATIVE
Entity-Feature pair: ('service', 'not good')



In [96]:
review = "Place was crowded, lacked ambience and not so tasty food at all."
sp = Sentiment_Parser(review)
sp.pprint()

	ORIGINAL TEXT: Place was crowded, lacked ambience and not so tasty food at all.

TEXT: Place was crowded
RESULT: NEUTRAL
Entity-Feature pair: ('Place', 'crowded')

TEXT: lacked ambience
RESULT: NEUTRAL
Entity-Feature pair: ('ambience', 'lacked')

TEXT: not so tasty food at all
RESULT: NEUTRAL
Entity-Feature pair: ('food', 'not tasty')



In [97]:
review = "Nice Place, a bit costly but recommended"
sp = Sentiment_Parser(review)
sp.pprint()

	ORIGINAL TEXT: Nice Place, a bit costly but recommended

TEXT: Nice Place
RESULT: POSITIVE
Entity-Feature pair: ('Nice,Place', '')

TEXT: a bit costly
RESULT: NEGATIVE
Entity-Feature pair: ('bit', 'costly')

TEXT: recommended
RESULT: POSITIVE
Entity-Feature pair: ('', 'recommended')



In [98]:
review = "Nice place for partying, friendly staffs and great food. Chicken Curry was damn tasty."
sp = Sentiment_Parser(review)
sp.pprint()

	ORIGINAL TEXT: Nice place for partying, friendly staffs and great food. Chicken Curry was damn tasty.

TEXT: Nice place for partying
RESULT: POSITIVE
Entity-Feature pair: ('place', 'Nice')

TEXT: friendly staffs
RESULT: POSITIVE
Entity-Feature pair: ('staffs', 'friendly')

TEXT: great food
RESULT: POSITIVE
Entity-Feature pair: ('food', 'great')

TEXT: Chicken Curry was damn tasty
RESULT: NEGATIVE
Entity-Feature pair: ('Chicken,Curry', 'tasty')

